In [1]:
import pandas as pd

In [2]:
file_name = "Resources/willamette_water.csv"

In [3]:
willamette_water_df = pd.read_csv(file_name, sep="\t")
willamette_water_df.head()

,agency_cd,site_no,datetime,tz_cd,water_temp,status_temp,oxygen,status_o2
0,USGS,14211720,01/22/2009 00:00,PST,14.1,A,4.6,A
1,USGS,14211720,01/22/2009 00:30,PST,14.2,A,4.7,A
2,USGS,14211720,01/22/2009 01:00,PST,14.2,A,4.7,A
3,USGS,14211720,01/22/2009 01:30,PST,14.1,A,4.7,A
4,USGS,14211720,01/22/2009 02:00,PST,14.1,A,4.7,A


In [4]:
willamette_water_df.dtypes

agency_cd       object
site_no          int64
datetime        object
tz_cd           object
water_temp     float64
status_temp     object
oxygen         float64
status_o2       object
dtype: object

In [5]:
willamette_water_df[willamette_water_df.isnull().values.any(axis=1)]

,agency_cd,site_no,datetime,tz_cd,water_temp,status_temp,oxygen,status_o2
1015,USGS,14211720,02/12/2009 03:30,PST,NaN,NaN,5.9,A
1026,USGS,14211720,02/12/2009 09:00,PST,NaN,NaN,5.8,A
9070,USGS,14211720,08/03/2009 00:00,PDT,9.9,A,NaN,NaN
9071,USGS,14211720,08/03/2009 00:30,PDT,9.6,A,NaN,NaN
9072,USGS,14211720,08/03/2009 01:00,PDT,9.4,A,NaN,NaN
...,...,...,...,...,...,...,...,...
192324,USGS,14211720,02/11/2020 09:00,PST,13.5,A,NaN,NaN
192325,USGS,14211720,02/11/2020 10:00,PST,NaN,NaN,7.0,A
194049,USGS,14211720,03/18/2020 09:00,PDT,12.0,A,NaN,NaN
203835,USGS,14211720,10/08/2020 09:30,PDT,NaN,NaN,16.6,P


In [6]:
#drop unnecessary columns
willamette_clean_df = willamette_water_df.dropna()
willamette_clean_df = willamette_clean_df.reset_index(drop=True)
willamette_clean_df

,agency_cd,site_no,datetime,tz_cd,water_temp,status_temp,oxygen,status_o2
0,USGS,14211720,01/22/2009 00:00,PST,14.1,A,4.6,A
1,USGS,14211720,01/22/2009 00:30,PST,14.2,A,4.7,A
2,USGS,14211720,01/22/2009 01:00,PST,14.2,A,4.7,A
3,USGS,14211720,01/22/2009 01:30,PST,14.1,A,4.7,A
4,USGS,14211720,01/22/2009 02:00,PST,14.1,A,4.7,A
...,...,...,...,...,...,...,...,...
203178,USGS,14211720,11/02/2020 16:00,PST,11.1,P,10.2,P
203179,USGS,14211720,11/02/2020 16:30,PST,11.1,P,10.2,P
203180,USGS,14211720,11/02/2020 17:00,PST,11.1,P,10.1,P
203181,USGS,14211720,11/02/2020 17:30,PST,11.0,P,10.2,P


In [7]:
willamette_reduce_df = willamette_clean_df.drop(columns=['agency_cd', 'site_no', 'tz_cd', 'status_temp', 'status_o2'])
willamette_reduce_df = willamette_reduce_df.reset_index(drop=True)
willamette_reduce_df.head()

,datetime,water_temp,oxygen
0,01/22/2009 00:00,14.1,4.6
1,01/22/2009 00:30,14.2,4.7
2,01/22/2009 01:00,14.2,4.7
3,01/22/2009 01:30,14.1,4.7
4,01/22/2009 02:00,14.1,4.7


In [8]:
willamette_reduce_df[['date','time']] = willamette_reduce_df['datetime'].str.split(' ', expand=True)
willamette_reduce_df = willamette_reduce_df.drop(columns=['datetime'])
willamette_reduce_df = willamette_reduce_df.reset_index(drop=True)
willamette_reduce_df.head()

,water_temp,oxygen,date,time
0,14.1,4.6,01/22/2009,00:00
1,14.2,4.7,01/22/2009,00:30
2,14.2,4.7,01/22/2009,01:00
3,14.1,4.7,01/22/2009,01:30
4,14.1,4.7,01/22/2009,02:00


In [9]:
willamette_reduce_df[['month','day','year']] = willamette_reduce_df['date'].str.split('/', expand=True)
willamette_reduce_df = willamette_reduce_df.drop(columns=['date'])
willamette_reduce_df.head()

,water_temp,oxygen,time,month,day,year
0,14.1,4.6,00:00,01,22,2009
1,14.2,4.7,00:30,01,22,2009
2,14.2,4.7,01:00,01,22,2009
3,14.1,4.7,01:30,01,22,2009
4,14.1,4.7,02:00,01,22,2009


In [16]:
willamette_reduce_df['month'] = willamette_reduce_df['month'].astype(int)
willamette_reduce_df['year'] = willamette_reduce_df['year'].astype(int)
willamette_reduce_df.dtypes

water_temp    float64
oxygen        float64
time           object
month           int32
day            object
year            int32
dtype: object

In [13]:
willamette_reduce_final= willamette_reduce_df[(willamette_reduce_df['month'] >= 3) & (willamette_reduce_df['month'] <= 5)]
willamette_reduce_final.head()

,water_temp,oxygen,time,month,day,year
1774,13.3,6.6,00:00,3,01,2009
1775,13.2,6.6,00:30,3,01,2009
1776,13.2,6.6,01:00,3,01,2009
1777,13.2,6.6,01:30,3,01,2009
1778,13.2,6.6,02:00,3,01,2009


In [19]:
willamette_reduce_final.groupby(['year','month']).mean()

water_temp     oxygen
year month                       
2009 3       12.694506   7.547288
     4       12.192917  10.010833
     5       11.587153  12.896875
2010 3       12.048053   9.332823
     4       12.043678  10.244612
     5       11.109684  12.550639
2011 3       13.233311   7.678869
     4       12.837716   9.299569
     5       11.867003  11.628495
2012 3       13.513459   7.079946
     4       12.853736   9.893750
     5       11.710013  12.718952
2013 3       12.094946   8.526415
     4       11.673889  11.144861
     5       10.590505  15.081481
2014 3       12.784522   8.942059
     4       11.806111  11.174722
     5       10.896774  14.487567
2015 3       11.525707  10.561238
     4       11.058038  12.457620
     5       10.001815  16.601344
2016 3       12.440473   9.163378
     4       10.983333  13.008472
     5       10.246397  15.701684
2017 3       13.156124   8.194213
     4       12.289360  10.112587
     5       11.358871  13.252823
2018 3       12.457604   7.991252
     4       12.028193  10.628402
     5       10.867295  15.574899
2019 3       12.707402   7.486676
     4       12.214585  10.634752
     5       10.424462  15.164785
2020 3       11.724966   8.992578
     4       11.300625  11.843403
     5       10.562030  14.671035